In [ ]:
import numpy as np
import pandas as pd

## Cost matrix

In [ ]:
#cost_matrix = pd.read_csv('../costs/costMatrixDistance.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial.csv', index_col=0)
cost_matrix = pd.read_csv('../costs/costMatrixDistance-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial-2020.csv', index_col=0)

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(float)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(str)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(str)

In [ ]:
cost_matrix

In [ ]:
# Number of shops to visit
n = cost_matrix.shape[0]

# Calculate M
M = 0
for i in range(n):
    for j in range(n):
        M += cost_matrix.iloc[i,j]

# M cost for the path i->i
for i in range(n):
    cost_matrix.iloc[i,i] = M + 1

In [ ]:
cost_matrix

## Shop demands

In [ ]:
# Select the shop locations and their demands
#shop_demands = pd.read_csv('../2015_shop_locations.csv').set_index('id')[['stage', 'demand(kg)']].astype(float)
shop_demands = pd.read_csv('../2020_shop_locations.csv').set_index('id')[['stage', 'demand(kg)']].astype(float)

shop_demands = shop_demands.loc[shop_demands['stage'].isin([1, 2, 3, 4]), 'demand(kg)'] # Choose the stages to visit
shops = shop_demands.index
nodes = shops.copy()
nodes = np.append('Depot', nodes)

## Starting solution

In [ ]:
# Start from the storage location and create a separate route for each shop

# Create a list of unvisited shops
unvisited = shop_demands.index.to_list()

# Calculate the cost of the route
cost = 0

# Van capacity
van_capacity = 300
remaining_capacity = van_capacity

In [ ]:
active_arcs = []  # List to store paths for all trucks

while len(unvisited) > 0:

    next_shop = unvisited[0]  # Get the first unvisited shop

    # Get the costs of the unvisited shops
    cost += cost_matrix.loc['Depot', next_shop]

    # Store the arc between current and next_shop
    active_arcs.append(('Depot', next_shop))
    active_arcs.append((next_shop, 'Depot'))

    # Remove the lowest cost shop from the unvisited list
    unvisited.remove(next_shop)

## Savings

In [ ]:
# Clarke-Wright Savings Algorithm
def calculate_savings(cost_matrix, depot='Depot'):
    """
    Calculate savings for merging routes in the vehicle routing problem.

    Parameters:
    - cost_matrix: DataFrame of transportation costs
    - depot: String identifier for the depot/storage location

    Returns:
    - Sorted list of savings opportunities
    """

    # Calculate savings for each pair of shops
    savings = []
    for i in range(len(shops)):
        for j in range(i+1, len(shops)):
            shop1, shop2 = shops[i], shops[j]

            # Calculate savings: cost(depot->i) + cost(depot->j) - cost(i->j)
            saving = (cost_matrix.loc[depot, shop1] +
                      cost_matrix.loc[depot, shop2] -
                      cost_matrix.loc[shop1, shop2])

            savings.append({
                'shop1': shop1,
                'shop2': shop2,
                'saving': saving
            })

    # Sort savings in descending order
    return sorted(savings, key=lambda x: x['saving'], reverse=True)

# Modify existing solution using savings algorithm
def apply_savings_algorithm(active_arcs, cost_matrix, shop_demands, van_capacity, depot='Depot'):
    """
    Apply Clarke-Wright savings algorithm to improve route efficiency.

    Parameters:
    - active_arcs: List of current routes
    - cost_matrix: DataFrame of transportation costs
    - shop_demands: Series of shop demands
    - van_capacity: Maximum vehicle capacity

    Returns:
    - Improved routes
    """
    # Calculate savings opportunities
    savings = calculate_savings(cost_matrix)

    # Create a route dictionary for easy manipulation
    routes = {}
    for route_idx, shop in enumerate(active_arcs):
        if shop[1] != depot:
            routes[route_idx] = [shop[1]]

    # Apply savings
    for saving in savings:
        shop1, shop2 = saving['shop1'], saving['shop2']

        # Find routes containing these shops
        route1_idx = route2_idx = None
        route1_pos = route2_pos = None

        for idx, route in routes.items():
            if shop1 in route:
                route1_idx = idx
                route1_pos = route.index(shop1)
            if shop2 in route:
                route2_idx = idx
                route2_pos = route.index(shop2)

        # Skip if shops are in the same route or no routes found
        if route1_idx is None or route2_idx is None or route1_idx == route2_idx:
            continue

        # Check capacity constraint
        route1_total_demand = sum(shop_demands[shop] for shop in routes[route1_idx])
        route2_total_demand = sum(shop_demands[shop] for shop in routes[route2_idx])

        # Merge routes if capacity allows
        if route1_total_demand + route2_total_demand <= van_capacity:
            # Merge routes
            if route1_pos == len(routes[route1_idx]) - 1 and route2_pos == 0:
                # Append route2 to route1
                routes[route1_idx].extend(routes[route2_idx])
                del routes[route2_idx]
            elif route2_pos == len(routes[route2_idx]) - 1 and route1_pos == 0:
                # Prepend route1 to route2
                routes[route2_idx] = routes[route1_idx] + routes[route2_idx]
                del routes[route1_idx]

    # Reconstruct routes with depot
    improved_routes = []
    for route in routes.values():
        improved_routes.append((depot, route[0]))
        improved_routes.extend([(route[i], route[i+1]) for i in range(len(route)-1)])
        improved_routes.append((route[-1], depot))

    return improved_routes

## Cost and route

In [ ]:
def calculate_route_cost(routes, cost_matrix):
    """
    Calculate the total cost of routes by summing arc costs from the cost matrix.

    Parameters:
    - routes: List of routes, where each route is a list of (start, end) tuples
    - cost_matrix: DataFrame of transportation costs

    Returns:
    - Total route cost
    """
    total_cost = 0
    for route in routes:
        total_cost += cost_matrix.loc[route[0], route[1]]
    return total_cost

In [ ]:
# Calculate cost of original routes
original_routes_cost = calculate_route_cost(active_arcs, cost_matrix)

# Apply the savings algorithm (from previous code)
improved_routes = apply_savings_algorithm(active_arcs, cost_matrix, shop_demands, van_capacity)


# Calculate cost of improved routes
improved_routes_cost = calculate_route_cost(improved_routes, cost_matrix)

# Print detailed route cost information
print("Original Routes:")
print("Number of routes:", len(active_arcs))
print("Total route cost:", original_routes_cost)

print("\nImproved Routes:")
print("Number of routes:", len(improved_routes))
print("Total route cost:", improved_routes_cost)

print("\nCost Savings:")
print("Absolute savings:", original_routes_cost - improved_routes_cost)
print("Percentage savings: {:.2f}%".format((original_routes_cost - improved_routes_cost) / original_routes_cost * 100))

In [ ]:
improved_routes

In [ ]:
# Convert the list of active arcs to a DataFrame
active_arcs_df = pd.DataFrame(improved_routes, columns=['From', 'To'])

# Save the DataFrame to a CSV file
active_arcs_df.to_csv('../routings/routing_savings_2020_stage_4_Distance.csv', index=False)